# Shift Planner Modeling Task
The goal is to help a very big restaurant, specialised in on-demand food delivery, to manage its fleet of drivers.

## Problem Statement
The restaurant receives multiple orders every day, and it relies on a sort of forecasting for the next day to estimate how many drivers are required every hour. For example:

Given this demand forecast we need to automatically build the shifts respecting certain constraints:

* Each shift need to be at least of 4h length.
* Each shift cannot be longer than 10h length.
* When we cover the hours of the demand, there might be the case in which we allocate more hours (oversupply). This is an acceptable behavior, but we need to minimise these oversupply hours.

Data
The sample data is available in this dropbox link. It contains some sample cases in JSON format, in which the input includes the forecasted demand and the configuration, such as:
```json
{
  "config": {
    "min_shift_hours": 4,
    "max_shift_hours": 10,
    "timeout_sec": 10
  },
  "timeslots": [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
  "demand": [0, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 1],
  "solution": {
    "shifts": []
  }
}
```

## Evaluation
The solution needs to include:

* __Mathematical formulation of the problem__ (to make it clear and understandable independently of the programming language used).
* __Implementation__ with the preferred programming language, with the instruction to run the given examples (and more).
* __Solutions__ of the forecasted demand shifts in the format defined in the Data section.

The evaluation will consider:

* __Correctness__ of the model described.
* Solution __performance__ (time/quality trade-off).

In case of doubts about the problem statement we require the candidate to make her/his assumptions without being blocked.

***

# Approach

It is clear that it is not feasible to store all possible combinations of sets of shifts (where each set of shifts consists on the list of shifts assigned to each driver) in order to consider those that satisfy the forecasted demand.

Nevertheless, we can model these sets of shifts in a much more efficient way: __constraint optimisation__ or __constraint programming__. The algorithm will only consider those combinations of shifts that satisfy the problem constraints (demand, shift minimum and maximun duration, etc).

## Formulation

We will create a shift matrix (list of lists):

```LaTeX
shifts[(i,j)] = {0, 1}
```

where `i` represents the driver `i` (`i = 0, ..., drivers`) and `j` represents the timeslot `j` (`j = 0, ..., timeslots`). If the driver `i` is working in the timeslot `j`, then `shifts[(i,j)] = 1`. If he is not working, then `shifts[(i,j)] = 0`.

The [constraint optimization](https://en.wikipedia.org/wiki/Constraint_programming) approach consists on filling up the `shifts` matrix with `0` or `1` satisfying all constraints.

## Constraints

### Demand

The number of drivers working in one timeslot has to be no less than the forecasted demand of drivers for that specific timeslot. We can write it as follows:

```LaTeX
for each timeslot j
    sum(shifts[(i,j)] for each driver i) >= demand[j]
```
### Shift duration

__Assumptions__

* We will consider that each driver only works in __one shift__ each day.
* Each shift need to be at least of `4h` length.
* Each shift cannot be longer than `10h` length.

Therefore:

```LaTeX
for each driver i
    sum(shifts[(i,j)] for each timeslot j) between (4, 10)
```
Further, we need to control that working timeslots are consecutive (only one shift per driver):

```LaTeX
for each driver i
    for each timeslot j
        shifts[(i,j)] == shifts[(i,j+1)] or shifts[(i,j+1)] == shifts[(i,j+2)]
```
This last condition will not properly work as expected since the algorithm can propose combinations such as:
```json
    [1,1,1,1,0,0,1,1,1,1]
```
and thus leaving a gap in the middle. In order to add extra control to this constraint we will write the following condition to be satisfied:
```LaTeX
for each driver i
    for each timeslot j
        shifts[(i,j)] == shifts[(i,j+2)] or shifts[(i,j+2)] == shifts[(i,j+4)]
```
Since the maximum number of timeslots provided is 13, there is no need to add more conditions to take care of these gaps.

Note: this is not exactly a constraint to be introduced to the model but a starting point to run these simulations. The minimum number of drivers (or shifts) is, at least, equal to the peak of demand forecasted.

## Objective

From all possible solutions, we need to pick the one that minimises the number of oversupply hours.

```LaTeX
min( sum(shifts) ) 
```

With all these ingredients we can resolve this constrained system of equations and inequations for different amount of drivers (=shifts) and find the optimal combination (trade-off between computing time and quality in terms of oversupply hours).

# Programming

For this exercise we will use [Google's Optimization Tools](https://developers.google.com/optimization/cp/) (`ortools`). Google Optimization Tools (a.k.a., OR-Tools) is an open-source, fast and portable software suite for solving combinatorial optimization problems.

There is a Constraint Programming Solver written in Python that we can borrow and suits perfectly our needs.

---

# Ex1

First we will check how well performs with the ex1:

In [1]:
import os
import json
import time
import pprint

pp = pprint.PrettyPrinter(indent=4)

# read files

path_to_json = 'planning_demand/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

print json_files

# load json data

with open(os.path.join(path_to_json, 'ex1-in.json')) as json_file:
    json_ex1_in = json.load(json_file)

['ex2-in.json', 'ex3-out.json', 'ex1-in.json', 'ex3-in.json', 'ex2-out.json', 'ex1-out.json']


In [2]:

## Creating the solver and shift matrix
## ------------------------------------------------------

from ortools.constraint_solver import pywrapcp

# Create the solver
solver = pywrapcp.Solver("ex1")

# Variables
num_vals = 2 # [0, 1] driver working or not
demand   = json_ex1_in['demand'] # demand forecast
slots    = len(demand) # number of timeslots
drivers  = 5 # number of drivers (at least max(demand))
min_h    = 4 # minimum number of hours per shift
max_h    = 10 # maximum number of hours per shift

# Create shift variables
shifts = {}
for j in range(drivers):
    for i in range(slots):
        shifts[(j,i)]=solver.IntVar(0, num_vals-1, "d%i_slot%i" % (j,i))

# flattened version of shifts (required by Phase method)
s_flat = [shifts[(j,i)] for j in range(drivers) for i in range(slots)]


## Constraints
## ------------------------------------------------------

# Demand by timeslot
for i in range(slots):
    solver.Add(solver.Sum([shifts[(j,i)] for j in range(drivers)]) >= demand[i])

# Maximum and minimum duration of shifts per driver
for j in range(drivers):
    solver.Add(solver.Sum([shifts[(j,i)] for i in range(slots)]) >= min_h)
    solver.Add(solver.Sum([shifts[(j,i)] for i in range(slots)]) <= max_h)

# Consecutive hours working (no gaps, one shift per driver)
for j in range(drivers):
    for i in range(slots-2):
        solver.Add(solver.Max(shifts[(j, i)] == shifts[(j, i+1)], shifts[(j, i+1)] == shifts[(j, i+2)]) == 1)
    for i in range(slots-4):
        solver.Add(solver.Max(shifts[(j, i)] == shifts[(j, i+2)], shifts[(j, i+2)] == shifts[(j, i+4)]) == 1)


## Find soltions and display results
## ------------------------------------------------------

# Create decision builder
db = solver.Phase(s_flat, 
                  solver.CHOOSE_FIRST_UNBOUND, 
                  solver.ASSIGN_MIN_VALUE)

solver.Solve(db)

solver.NextSolution()

# Print results
print "\nForecasted demand"
print demand
print ""
for j in range(drivers):
    t = []
    for i in range(slots):
        t.append(shifts[(j,i)].Value())
    print(t)

t = []
for i in range(slots):
    ssum = 0
    for j in range(drivers):
        ssum += shifts[(j,i)].Value()
    t.append(ssum)

print "\nDrivers per timeslot"
print t


Forecasted demand
[0, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 1]

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]

Drivers per timeslot
[0, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 1]


Now I can wrap all this code within a function and pass in the desired number of drivers (=shifts) and exercise to solve.

Default parameters / options:

* The time limit for finding a solution is `15 seconds`.

In [3]:
import os
import json
from ortools.constraint_solver import pywrapcp

def load_data(exercise):
    """Load JSON data for a given exercise"""

    json_files = ['ex1-in.json', 'ex2-in.json', 'ex3-in.json']
    path_to_json = 'planning_demand/'

    if exercise == "ex1":
        with open(os.path.join(path_to_json, json_files[0])) as json_file:
            json_data = json.load(json_file)
    elif exercise == "ex2":
        with open(os.path.join(path_to_json, json_files[1])) as json_file:
            json_data = json.load(json_file)
    elif exercise == "ex3":
        with open(os.path.join(path_to_json, json_files[2])) as json_file:
            json_data = json.load(json_file)

    return json_data


def optimize_shifts(exercise, drivers):

    ## Creating the solver and shift matrix
    ## ------------------------------------------------------

    # Create the solver.
    solver = pywrapcp.Solver(exercise)

    print "\n", "*"*100, "\nExercise %s \n" % exercise[-1], "*"*100

    # Set Variables.
    num_vals  = 2 # [0, 1] driver working or not
    json_data = load_data(exercise)
    demand    = json_data['demand'] # demand forecast
    slots     = len(demand) # number of timeslots
    min_h     = 4 # minimum number of hours per shift
    max_h     = 10 # maximum number of hours per shift

    print "\nNumber of drivers:", drivers

    # Create shift variables.
    shifts = {}
    for j in range(drivers):
        for i in range(slots):
            shifts[(j,i)]=solver.IntVar(0, num_vals-1, "d%i_slot%i" % (j,i))

    # Flattened version of shifts (required by Phase method).
    s_flat = [shifts[(j,i)] for j in range(drivers) for i in range(slots)]


    ## Constraints
    ## ------------------------------------------------------

    # Demand by timeslot.
    for i in range(slots):
        solver.Add(solver.Sum([shifts[(j,i)] for j in range(drivers)]) >= demand[i])

    # Maximum and minimum duration of shifts per driver.
    for j in range(drivers):
        solver.Add(solver.Sum([shifts[(j,i)] for i in range(slots)]) >= min_h)
        solver.Add(solver.Sum([shifts[(j,i)] for i in range(slots)]) <= max_h)

    # Consecutive hours working (no gaps, one shift per driver).
    for j in range(drivers):
        for i in range(slots-2):
            solver.Add(solver.Max(shifts[(j, i)] == shifts[(j, i+1)], shifts[(j, i+1)] == shifts[(j, i+2)]) == 1)
        for i in range(slots-4):
            solver.Add(solver.Max(shifts[(j, i)] == shifts[(j, i+2)], shifts[(j, i+2)] == shifts[(j, i+4)]) == 1)


    # Define objective function
    ## ------------------------------------------------------

    obj_var = solver.Sum([shifts[(j,i)]for j in range(drivers) for i in range(slots)])
    obj_monitor = solver.Minimize(obj_var, 1)
 

    ## Find soltions and display results
    ## ------------------------------------------------------

    # Create decision builder.
    db = solver.Phase(s_flat, 
                      solver.CHOOSE_FIRST_UNBOUND, 
                      solver.ASSIGN_MIN_VALUE)
    
    # Set time limit.
    time_limit = 15000
    time_limit_ms = solver.TimeLimit(time_limit)

    # Add variables to the solution collector.
    solution = solver.Assignment()
    solution.Add(s_flat)
    collector = solver.FirstSolutionCollector() # AllSolutionCollector
    collector.Add(s_flat)
    collector.AddObjective(obj_var)

    solver.Solve(db, [obj_monitor, time_limit_ms, collector])

    if solver.Solve(db, [time_limit_ms, collector]):
        print "Total demand hours:", sum(demand)
        print "Optimal amount of estimated hours:", collector.ObjectiveValue(0)
        print "Solutions found in %s seconds: " % str(time_limit/1000), collector.SolutionCount()
        print "Time:", solver.WallTime(), "ms"
        print "\n", "*"*100, "\n"
    
        # print shifts
        for j in range(drivers):
            l = []
            for i in range(slots):
                l.append(collector.Value(0, shifts[(j, i)]))
            print l

        t = []
        for i in range(slots):
            l = 0
            for j in range(drivers):
                l +=collector.Value(0, shifts[(j, i)])
            t.append(l)
        print "\nEstimated drivers per timeslot:"
        print t
        print "\nForecasted demand:"
        print demand

        print "\nOversupply hours:", str(collector.ObjectiveValue(0)), "-", str(sum(demand)), "=", str(collector.ObjectiveValue(0)-sum(demand))

    else:
        print "\n", "*"*100, "\n"
        print "No solution found. Time limit set to %s seconds." % str(time_limit/1000)

    print ""

# Ex2

We are going to use the `optimize_shifts()` function to minimize the number of oversupply hours for the forecasted demand. We will start the iterative process with 38 drivers (=shifts) and substract one in each iteration until no solution is found.

In [4]:
print "\nTimeslots:","\n",load_data("ex2")['timeslots']
print "\nForecasted demand:","\n", load_data("ex2")['demand'], "\n"


Timeslots: 
[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

Forecasted demand: 
[0, 9, 9, 9, 16, 16, 17, 17, 17, 17, 17, 17, 6] 



In [5]:
for i in range(38, 30, -1):
    optimize_shifts("ex2", i)


**************************************************************************************************** 
Exercise 2 
****************************************************************************************************

Number of drivers: 38
Total demand hours: 167
Optimal amount of estimated hours: 195
Solutions found in 15 seconds:  1
Time: 15069 ms

**************************************************************************************************** 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 

Total demand hours: 167
Optimal amount of estimated hours: 179
Solutions found in 15 seconds:  1
Time: 15030 ms

**************************************************************************************************** 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 1,

It seems that we need 34 drivers (=shifts) at least. With such set of shifts we are only oversupplying by 12 hours (179 hours).

# Ex3

In [6]:
print "\nTimeslots:","\n",load_data("ex3")['timeslots']
print "\nForecasted demand:","\n", load_data("ex3")['demand'], "\n"


Timeslots: 
[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

Forecasted demand: 
[35, 35, 35, 50, 50, 50, 50, 50, 40, 30, 30, 10, 10] 



In this case we will need way more drivers (=shifts). I run a few iterations to approximately find the optimal number of drivers, which is around to 80.

In [7]:
for i in range(81, 78, -1):
    optimize_shifts("ex3", i)


**************************************************************************************************** 
Exercise 3 
****************************************************************************************************

Number of drivers: 81
Total demand hours: 475
Optimal amount of estimated hours: 519
Solutions found in 15 seconds:  1
Time: 15077 ms

**************************************************************************************************** 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 

# Results

These are the optimal solutions for each exercise:

| Exercise    | Demand (hours)  | Drivers/Shifts needed | Oversupply hours  |
| :---------- | --------------: | --------------------: | ----------------: |
| Ex1         | 35              | 5                     | 0                 |
| Ex2         | 167             | 34                    | 12                |
| Ex3         | 475             | 80                    | 40                |


# Dump data to JSON

If we try to return data from `optimize_shifts()` method we run into a "Segmentation Fault" error. This happens when a python extension (written in C) tries to access a memory beyond reach. In order to dodge this error I will rewrite a lightweight version of the `optimize_shifts()` function to build the output JSON files.

In [8]:
def dump_shifts(exercise, drivers):

    ## Creating the solver and shift matrix
    ## ------------------------------------------------------

    # Create the solver.
    solver = pywrapcp.Solver(exercise)

    # Set Variables.
    num_vals  = 2 # [0, 1] driver working or not
    json_data = load_data(exercise)
    demand    = json_data['demand'] # demand forecast
    slots     = len(demand) # number of timeslots
    min_h     = 4 # minimum number of hours per shift
    max_h     = 10 # maximum number of hours per shift

    # Create shift variables.
    shifts = {}
    for j in range(drivers):
        for i in range(slots):
            shifts[(j,i)]=solver.IntVar(0, num_vals-1, "d%i_slot%i" % (j,i))

    # Flattened version of shifts (required by Phase method).
    s_flat = [shifts[(j,i)] for j in range(drivers) for i in range(slots)]


    ## Constraints
    ## ------------------------------------------------------

    # Demand by timeslot.
    for i in range(slots):
        solver.Add(solver.Sum([shifts[(j,i)] for j in range(drivers)]) >= demand[i])

    # Maximum and minimum duration of shifts per driver.
    for j in range(drivers):
        solver.Add(solver.Sum([shifts[(j,i)] for i in range(slots)]) >= min_h)
        solver.Add(solver.Sum([shifts[(j,i)] for i in range(slots)]) <= max_h)

    # Consecutive hours working (no gaps, one shift per driver).
    for j in range(drivers):
        for i in range(slots-2):
            solver.Add(solver.Max(shifts[(j, i)] == shifts[(j, i+1)], shifts[(j, i+1)] == shifts[(j, i+2)]) == 1)
        for i in range(slots-4):
            solver.Add(solver.Max(shifts[(j, i)] == shifts[(j, i+2)], shifts[(j, i+2)] == shifts[(j, i+4)]) == 1)


    # Define objective function
    ## ------------------------------------------------------

    obj_var = solver.Sum([shifts[(j,i)]for j in range(drivers) for i in range(slots)])
    obj_monitor = solver.Minimize(obj_var, 1)
 

    ## Find soltions and display results
    ## ------------------------------------------------------

    # Create decision builder.
    db = solver.Phase(s_flat, 
                      solver.CHOOSE_FIRST_UNBOUND, 
                      solver.ASSIGN_MIN_VALUE)
    
    # Set time limit.
    time_limit = 15000
    time_limit_ms = solver.TimeLimit(time_limit)

    # Add variables to the solution collector.
    solution = solver.Assignment()
    solution.Add(s_flat)
    collector = solver.FirstSolutionCollector() # AllSolutionCollector
    collector.Add(s_flat)
    collector.AddObjective(obj_var)

    solver.Solve(db, [obj_monitor, time_limit_ms, collector])

    if solver.Solve(db, [time_limit_ms, collector]):
   
        shifts_list = []
        for j in range(drivers):
            l = []
            for i in range(slots):
                l.append(collector.Value(0, shifts[(j, i)]))
            shifts_list.append(l)
            
        return shifts_list, json_data



In [9]:
shifts, json_data = dump_shifts("ex1", 5)

json_data['solution']['shifts'] = shifts

pp.pprint(json_data)

with open('./ex1-out.json', 'w') as outfile:
    json.dump(json_data, outfile)

{   u'config': {   u'max_shift_hours': 10,
                   u'min_shift_hours': 4,
                   u'timeout_sec': 10},
    u'demand': [0, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 1],
    u'solution': {   u'shifts': [   [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
                                    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
                                    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
                                    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
                                    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]},
    u'timeslots': [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]}


In [10]:
shifts, json_data = dump_shifts("ex2", 34)

json_data['solution']['shifts'] = shifts

pp.pprint(json_data)

with open('./ex2-out.json', 'w') as outfile:
    json.dump(json_data, outfile)

{   u'config': {   u'max_shift_hours': 10,
                   u'min_shift_hours': 4,
                   u'timeout_sec': 10},
    u'demand': [0, 9, 9, 9, 16, 16, 17, 17, 17, 17, 17, 17, 6],
    u'solution': {   u'shifts': [   [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0

In [11]:
shifts, json_data = dump_shifts("ex3", 80)

json_data['solution']['shifts'] = shifts

pp.pprint(json_data)

with open('./ex3-out.json', 'w') as outfile:
    json.dump(json_data, outfile)

{   u'config': {   u'max_shift_hours': 10,
                   u'min_shift_hours': 4,
                   u'timeout_sec': 10},
    u'demand': [35, 35, 35, 50, 50, 50, 50, 50, 40, 30, 30, 10, 10],
    u'solution': {   u'shifts': [   [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
                                    

# Next steps ...

* For new demand forecasts it would be necessary to automate the process of finding the optimal amount of drivers (shifts)
* Fine tunning of CP Solver to make propagations more efficient